# Matrix Factorization

In [38]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()


    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    
    def loss(self):
        L = 0 
        for i in range(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))

    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.values


rs = MF(Y_data, K = 2, max_iter = 1000, print_every = 1000)

rs.fit()
rs.pred(6, 1)
rs.pred_for_user(2)

iter = 1000 , loss = 0.508463130259286 , RMSE train = 0.7957566106575795


[(1, 2.216147590028831)]

In [2]:
print (rs.X.dot(rs.W) + rs.mu)

[[4.16628879 4.01679173 1.80739216 0.2216641  1.71831781 1.43298887
  2.37172342]
 [3.65911475 3.28635726 2.21623444 0.81559029 2.14422556 1.47954007
  2.93825388]
 [3.99831474 3.85487775 1.87312334 0.47687437 1.87786482 1.42253559
  2.46563305]
 [2.44208656 1.70075899 3.05171113 2.36303964 3.20487695 1.53718673
  4.10187496]
 [2.26009497 1.32027645 3.30151439 2.48959449 3.33035137 1.59217099
  4.44384398]]


# Áp dụng lên MovieLens 100k

In [39]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [41]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nUser-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.608456055171417 , RMSE train = 1.2045190502583125
iter = 20 , loss = 2.620172123308861 , RMSE train = 1.0381442380147747
iter = 30 , loss = 1.3345349557492776 , RMSE train = 1.0296039671410036
iter = 40 , loss = 0.7487863430662083 , RMSE train = 1.029228595765795
iter = 50 , loss = 0.48035335491037856 , RMSE train = 1.0292139958405766
iter = 60 , loss = 0.3572642933185018 , RMSE train = 1.0292140409918502
iter = 70 , loss = 0.3008184520435929 , RMSE train = 1.0292142451508761
iter = 80 , loss = 0.2749334349176179 , RMSE train = 1.0292143086569905
iter = 90 , loss = 0.26306302201137305 , RMSE train = 1.0292143254374069
iter = 100 , loss = 0.2576194554003539 , RMSE train = 1.0292143296466596

User-based MF, RMSE = 1.0603799058767178


In [44]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.671025954048728 , RMSE train = 1.1832667011013596
iter = 20 , loss = 2.634862298100188 , RMSE train = 1.0057441000059497
iter = 30 , loss = 1.3319264652442475 , RMSE train = 0.9965683821201389
iter = 40 , loss = 0.7384098666674402 , RMSE train = 0.9961887080182692
iter = 50 , loss = 0.46641297792079384 , RMSE train = 0.9961789520315569
iter = 60 , loss = 0.34168738420448147 , RMSE train = 0.9961802640629733
iter = 70 , loss = 0.28449041613013043 , RMSE train = 0.996180744350654
iter = 80 , loss = 0.25826075779697444 , RMSE train = 0.9961808660908547
iter = 90 , loss = 0.24623223639279657 , RMSE train = 0.9961808950702287
iter = 100 , loss = 0.24071614068807604 , RMSE train = 0.9961809018314804

Item-based MF, RMSE = 1.0486424594189638


In [43]:
rs = MF(rate_train, K = 10, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

iter = 10 , loss = 4.849780587747066 , RMSE train = 1.898039853041211
iter = 20 , loss = 4.505330133233762 , RMSE train = 1.880891443514488
iter = 30 , loss = 4.201314876559252 , RMSE train = 1.8638775269412833
iter = 40 , loss = 3.93143191636852 , RMSE train = 1.8470659652779027
iter = 50 , loss = 3.6905819976332874 , RMSE train = 1.8304132879111246
iter = 60 , loss = 3.4746005929214925 , RMSE train = 1.8139119455872574
iter = 70 , loss = 3.2800571872216637 , RMSE train = 1.7976176040009375
iter = 80 , loss = 3.104103548130094 , RMSE train = 1.7816147091227068
iter = 90 , loss = 2.944357683106338 , RMSE train = 1.7658886402865426
iter = 100 , loss = 2.7988141367812904 , RMSE train = 1.750493767821328

Item-based MF, RMSE = 1.8458766402915951


In [55]:
RMSE = rs.evaluate_RMSE(rate_train)

In [56]:
print (RMSE)

1.43478611596
